In [83]:
import xgboost as xgb
import pandas as pd
import numpy as np



In [84]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
total=pd.concat([train,test])
print str(len(train))+":"+str(len(test))
total.info()

891:418
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [85]:
total.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [86]:
#train[['Sex','Survived']].groupby(['Sex'], as_index=False).mean()
total['FamilySize']=total['SibSp']+total['Parch']+1
#total[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean()

,FamilySize,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


In [101]:
total['IsAlone']=1
total.loc[total['FamilySize']>1, 'IsAlone']=0
#total[['IsAlone','Survived']].groupby(['IsAlone'], as_index=False).mean()

In [102]:
total['Embarked'].fillna('S')
total[['Embarked','Survived']].groupby('Embarked', as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [107]:
total['Fare'].fillna(total['Fare'].median())
total['FareCategorical']=pd.qcut(total['Fare'],4)
#total[['FareCategorical','Survived']].groupby('FareCategorical', as_index=False).mean()

In [118]:
mu=total['Age'].mean()
delta=total['Age'].std()
rand_list=np.random.randint(mu-delta,mu+delta,size=total['Age'].isnull().sum())
total.loc[total['Age'].isnull(), 'Age']=rand_list
total['Age']=total['Age'].astype(int)
total['AgeCategorical']=pd.cut(total['Age'],5)
total[['AgeCategorical','Survived']].groupby('AgeCategorical', as_index=False).mean()

,AgeCategorical,Survived
0,"(-0.08, 16.0]",0.504425
1,"(16.0, 32.0]",0.361798
2,"(32.0, 48.0]",0.367589
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909
